# Some ways to use the Responses API in Llama Stack

The development of this notebook was assisted by Google Gemini and Cursor using Claude 4 Sonnet.

Before getting started, follow the following steps.

First install Llama Stack and all of the other dependencies for this notebook.
One way to do that is:

- First install Python 3.12 or later (do not try this with older versions of Python: it will not work).
- Then make a Python virtual environment.
- Then within that virtual environment run:

```
pip install -r requirements.txt
```

Once everything is installed, run the Llama Stack server:

```
llama stack run run.yaml --image-type venv
```

Also run the National Parks Service Model Context Protocol (MCP) server:

```
python nps_mcp_server.py --transport sse --port 3005
```

See [README_NPS](../README_NPS.md) for more information about this server.

## Configuration

Here we point to the locations of the servers we just started up above.  Also, we provide the model ID for the model we want to use.  The model ID should be one that that is specified in [run.yaml](./run.yaml).  In the [run.yaml](./run.yaml) included here, we have the following models defined:

- `gpt-3.5-turbo` and `gpt4o` are models from OpenAI.  They will only work if you have OPENAI_API_KEY set in your environment to a [valid OpenAI API key](https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key).
- `Llama-3.3-70B-Instruct` is a a model from Meta Llama.  This will only work if you have an API key for the hosted [Llama API](https://www.llama.com/products/llama-api/).
- `meta-llama/llama-3-3-70b-instruct` is the same model running on watsonx.ai (which has a somewhat different style for model IDs).  This will only work if you have a WATSONX_API_KEY and WATSONX_PROJECT_ID set (which requires an IBM Cloud account).  You may also need to set WATSONX_BASE_URL set if your watsonx.ai instance is running anywhere other than US South (which is the default).

If you can't or don't want to get any of those API keys, you can update [run.yaml](./run.yaml) to use [another inference provider](https://llama-stack.readthedocs.io/en/latest/providers/inference/index.html#overview).  Llama Stack includes numerous providers for calling hosted models like the ones above.  It also includes providers to call models that you deploy and run yourself using a model serving capability, e.g., the [vLLM provider](https://llama-stack.readthedocs.io/en/latest/providers/inference/remote_vllm.html) or the [ollama provider](https://llama-stack.readthedocs.io/en/latest/providers/inference/remote_ollama.html).

In [1]:
LLAMA_STACK_URL = "http://localhost:8321/"
NPS_MCP_URL = "http://localhost:3005/sse/"
LLAMA_STACK_MODEL_ID = "gpt-4o"

## Using the Llama Stack client

The most obvious way to use the Responses API in Llama Stack is via the Llama Stack client.  That's the way that is most seamlessly integrated with Llama Stack itself, so we would recommend it for most beginning users who don't already have a commitment to another client library.  Here are some examples using this approach.

In [2]:
from llama_stack_client import LlamaStackClient

client = LlamaStackClient(base_url="http://localhost:8321")

### Plain model inference

We will start with a simple but powerful use case in which the generative AI model provides a response directly without needing extra content or tools:

In [3]:
simple_llama_stack_client_response = client.responses.create(
    model=LLAMA_STACK_MODEL_ID,
    input="What is the capital of France?"
)

simple_llama_stack_client_response

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"


ResponseObject(id='resp-535a913e-7373-4ee8-b620-28678e7364f1', created_at=1755107137, model='gpt-4o', object='response', output=[OutputOpenAIResponseMessage(content=[OutputOpenAIResponseMessageContentUnionMember2(annotations=[], text='The capital of France is Paris.', type='output_text')], role='assistant', type='message', id='msg_f30502f4-f37e-4449-9f08-7d8b46184761', status='completed')], parallel_tool_calls=False, status='completed', text=Text(format=TextFormat(type='text', description=None, name=None, schema_=None, strict=None)), error=None, previous_response_id=None, temperature=None, top_p=None, truncation=None, user=None)

The response object is a little complex to read, so we provide a function to print it out in a more readable format:

In [4]:
def print_simple_response(response):
    print(f"ID: {response.id}")
    print(f"Status: {response.status}")
    print(f"Model: {response.model}")
    print(f"Created at: {response.created_at}")
    print(f"Output type: {response.output[0].type}")
    print(f"Response content: {response.output[0].content[0].text}")

In [5]:
print_simple_response(simple_llama_stack_client_response)

ID: resp-535a913e-7373-4ee8-b620-28678e7364f1
Status: completed
Model: gpt-4o
Created at: 1755107137
Output type: message
Response content: The capital of France is Paris.


### Retrieval-Augmented Generation

Next we will expand out to enabling the model to use context from a document to complement its own internal knowledge.

To keep the example simple, we will just get one document, specifically a PDF file describing the US National Parks.

In [6]:
import requests
from pathlib import Path

# Download a sample PDF for demonstration
def download_sample_pdf(url: str, filename: str) -> str:
    """Download a PDF from URL and save it locally"""
    print(f"Downloading PDF from: {url}")
    response = requests.get(url)
    response.raise_for_status()
    
    filepath = Path(filename)
    with open(filepath, 'wb') as f:
        f.write(response.content)
    
    print(f"PDF saved as: {filepath}")
    return str(filepath)

pdf_url = "https://www.nps.gov/aboutus/upload/NPIndex2012-2016.pdf"
pdf_path = download_sample_pdf(pdf_url, "NPIndex2012-2016.pdf")
pdf_title = "The National Parks: Index 2012-2016"

PDF saved as: NPIndex2012-2016.pdf


Then we create a vector store and load the PDF file into that vector store.

In [7]:
import uuid

vector_store_name= f"vec_{str(uuid.uuid4())[0:8]}"

vector_store = client.vector_stores.create(name=vector_store_name)
vector_store_id = vector_store.id

vector_store_id

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/vector_stores "HTTP/1.1 200 OK"


'vs_3bd59d59-2706-4165-8a95-dd3599ab988c'

In [8]:
file_create_response = client.files.create(file=Path(pdf_path), purpose="assistants")
file_create_response

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/files "HTTP/1.1 200 OK"


File(id='file-02da3aef43cf435f92e1b913bbd3ddc4', bytes=6540612, created_at=1755107138, expires_at=1786643138, filename='NPIndex2012-2016.pdf', object='file', purpose='assistants')

In [9]:
file_ingest_response = client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_create_response.id,
)
file_ingest_response

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/vector_stores/vs_3bd59d59-2706-4165-8a95-dd3599ab988c/files "HTTP/1.1 200 OK"


VectorStoreFile(id='file-02da3aef43cf435f92e1b913bbd3ddc4', attributes={}, chunking_strategy=ChunkingStrategyVectorStoreChunkingStrategyAuto(type='auto'), created_at=1755107138, object='vector_store.file', status='completed', usage_bytes=0, vector_store_id='vs_3bd59d59-2706-4165-8a95-dd3599ab988c', last_error=None)

Now we can uses the Responses API with that vector store to get answers from that PDF file.

In [10]:
rag_llama_stack_client_response = client.responses.create(
    model=LLAMA_STACK_MODEL_ID,
    input="When did the Bering Land Bridge become a national preserve?",
    tools=[
        {
            "type": "file_search",
            "vector_store_ids": [vector_store_id],
        }
    ]
)

rag_llama_stack_client_response

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"


ResponseObject(id='resp-5f7287ac-571c-4be7-9279-00def69ec4cf', created_at=1755107165, model='gpt-4o', object='response', output=[OutputOpenAIResponseOutputMessageFileSearchToolCall(id='call_La0PfMIn1XoKXI5NQojXqWym', queries=['Bering Land Bridge national preserve designation date'], status='completed', type='file_search_call', results=[{'file_id': '', 'filename': '', 'text': ' for abundant wildlife and sport\n \nfishing for five species of salmon.\n \nEstablished Dec. 2, 1980. Length: 67 miles.\n \nAcreage—30,664.79 Federal: 26,417.85 \nNonfederal: 4,246.94. \nAniakchak  \nN\national Monument  and \nAniakchak  \nNational Preserve \n1000 Silver Street, Bldg.603 \nP\nO Box 245 \nKing Salmon, AK 99613 \n907\xad246\xad3305 \nwww.nps.gov/ania \nThe Aniakchak Caldera, covering some 30 square miles, is \no\nne of the great dry calderas in the world. Located in the \nvolcanically active Aleutian Mountains, the Aniakchak last \nerupted in 1931. The crater includes lava flows, cinder cones, \nan

This one is a little more complicated, so we will need a more powerful print method to see it all nicely formatted:

In [11]:
def print_rag_response(response):
    print(f"ID: {response.id}")
    print(f"Status: {response.status}")
    print(f"Model: {response.model}")
    print(f"Created at: {response.created_at}")
    print(f"Output items: {len(response.output)}")
    
    for i, output_item in enumerate(response.output):
        if len(response.output) > 1:
            print(f"\n--- Output Item {i+1} ---")
        print(f"Output type: {output_item.type}")
        
        if output_item.type in ("text", "message"):
            print(f"Response content: {output_item.content[0].text}")
        elif output_item.type == "file_search_call":
            print(f"  Tool Call ID: {output_item.id}")
            print(f"  Tool Status: {output_item.status}")
            # 'queries' is a list, so we join it for clean printing
            print(f"  Queries: {', '.join(output_item.queries)}")
            # Display results if they exist, otherwise note they are empty
            print(f"  Results: {output_item.results if output_item.results else 'None'}")
        else:
            print(f"Response content: {output_item.content}")

In [12]:
print_rag_response(rag_llama_stack_client_response)

ID: resp-5f7287ac-571c-4be7-9279-00def69ec4cf
Status: completed
Model: gpt-4o
Created at: 1755107165
Output items: 2

--- Output Item 1 ---
Output type: file_search_call
  Tool Call ID: call_La0PfMIn1XoKXI5NQojXqWym
  Tool Status: completed
  Queries: Bering Land Bridge national preserve designation date
  Results: [{'file_id': '', 'filename': '', 'text': ' for abundant wildlife and sport\n \nfishing for five species of salmon.\n \nEstablished Dec. 2, 1980. Length: 67 miles.\n \nAcreage—30,664.79 Federal: 26,417.85 \nNonfederal: 4,246.94. \nAniakchak  \nN\national Monument  and \nAniakchak  \nNational Preserve \n1000 Silver Street, Bldg.603 \nP\nO Box 245 \nKing Salmon, AK 99613 \n907\xad246\xad3305 \nwww.nps.gov/ania \nThe Aniakchak Caldera, covering some 30 square miles, is \no\nne of the great dry calderas in the world. Located in the \nvolcanically active Aleutian Mountains, the Aniakchak last \nerupted in 1931. The crater includes lava flows, cinder cones, \nand explosion pits, as

### MCP tool calling

Now we will move on to a more substantial example using the NPS MCP server you set up at the top of this notebook.

In [13]:
mcp_llama_stack_client_response = client.responses.create(
    model=LLAMA_STACK_MODEL_ID,
    input="Tell me about some parks in Rhode Island, and let me know if there are any upcoming events at them.",
    tools=[
        {
            "type": "mcp",
            "server_url": NPS_MCP_URL,
            "server_label": "National Parks Service tools",
        }
    ]
)

mcp_llama_stack_client_response

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"


ResponseObject(id='resp-4bddbed7-3503-4231-bd17-5beb63ee08b3', created_at=1755107168, model='gpt-4o', object='response', output=[OutputOpenAIResponseOutputMessageMcpListTools(id='mcp_list_5d571da3-1551-44e3-890d-2d1fff59958c', server_label='National Parks Service tools', tools=[OutputOpenAIResponseOutputMessageMcpListToolsTool(input_schema={'type': 'object', 'properties': {'state_code': {'type': 'string', 'description': ''}, 'park_code': {'type': 'string', 'description': ''}, 'query': {'type': 'string', 'description': ''}, 'limit': {'type': 'integer', 'description': ''}}, 'required': ['state_code', 'park_code', 'query', 'limit']}, name='search_parks', description="Search for national parks by state, park code, or query string.\n\nArgs:\n    state_code: Two-letter state code (e.g., 'CA', 'NY')\n    park_code: Four-letter park code (e.g., 'yell', 'acad')\n    query: Search query for park names or descriptions\n    limit: Maximum number of results to return (default: 10)\n\nReturns:\n    

The output for this one is even more complicated, so we'll add some more clauses to the print method to come up with a version that handles both RAG and MCP tool calling:

In [14]:
def print_response(response):
    print(f"ID: {response.id}")
    print(f"Status: {response.status}")
    print(f"Model: {response.model}")
    print(f"Created at: {response.created_at}")
    print(f"Output items: {len(response.output)}")
    
    for i, output_item in enumerate(response.output):
        if len(response.output) > 1:
            print(f"\n--- Output Item {i+1} ---")
        print(f"Output type: {output_item.type}")
        
        if output_item.type in ("text", "message"):
            print(f"Response content: {output_item.content[0].text}")
        elif output_item.type == "file_search_call":
            print(f"  Tool Call ID: {output_item.id}")
            print(f"  Tool Status: {output_item.status}")
            # 'queries' is a list, so we join it for clean printing
            print(f"  Queries: {', '.join(output_item.queries)}")
            # Display results if they exist, otherwise note they are empty
            print(f"  Results: {output_item.results if output_item.results else 'None'}")
        elif output_item.type == "mcp_list_tools":
            print_mcp_list_tools(output_item)
        elif output_item.type == "mcp_call":
            print_mcp_call(output_item)
        else:
            print(f"Response content: {output_item.content}")

def print_mcp_call(mcp_call):
    """Print MCP call in a nicely formatted way"""
    print(f"\n🛠️  MCP Tool Call: {mcp_call.name}")
    print(f"   Server: {mcp_call.server_label}")
    print(f"   ID: {mcp_call.id}")
    print(f"   Arguments: {mcp_call.arguments}")
    
    if mcp_call.error:
        print("Error: {mcp_call.error}")
    elif mcp_call.output:
        print("Output:")
        # Try to format JSON output nicely
        try:
            import json
            parsed_output = json.loads(mcp_call.output)
            print(json.dumps(parsed_output, indent=4))
        except:
            # If not valid JSON, print as-is
            print(f"   {mcp_call.output}")
    else:
        print("   ⏳ No output yet")

def print_mcp_list_tools(mcp_list_tools):
    """Print MCP list tools in a nicely formatted way"""
    print(f"\n🔧 MCP Server: {mcp_list_tools.server_label}")
    print(f"   ID: {mcp_list_tools.id}")
    print(f"   Available Tools: {len(mcp_list_tools.tools)}")
    print("=" * 80)
    
    for i, tool in enumerate(mcp_list_tools.tools, 1):
        print(f"\n{i}. {tool.name}")
        print(f"   Description: {tool.description}")
        
        # Parse and display input schema
        schema = tool.input_schema
        if schema and 'properties' in schema:
            properties = schema['properties']
            required = schema.get('required', [])
            
            print("   Parameters:")
            for param_name, param_info in properties.items():
                param_type = param_info.get('type', 'unknown')
                param_desc = param_info.get('description', 'No description')
                required_marker = " (required)" if param_name in required else " (optional)"
                print(f"     • {param_name} ({param_type}){required_marker}")
                if param_desc:
                    print(f"       {param_desc}")
        
        if i < len(mcp_list_tools.tools):
            print("-" * 40)

In [15]:
print_response(mcp_llama_stack_client_response)

ID: resp-4bddbed7-3503-4231-bd17-5beb63ee08b3
Status: completed
Model: gpt-4o
Created at: 1755107168
Output items: 8

--- Output Item 1 ---
Output type: mcp_list_tools

🔧 MCP Server: National Parks Service tools
   ID: mcp_list_5d571da3-1551-44e3-890d-2d1fff59958c
   Available Tools: 5

1. search_parks
   Description: Search for national parks by state, park code, or query string.

Args:
    state_code: Two-letter state code (e.g., 'CA', 'NY')
    park_code: Four-letter park code (e.g., 'yell', 'acad')
    query: Search query for park names or descriptions
    limit: Maximum number of results to return (default: 10)

Returns:
    JSON string with park information including name, description, website, and location
   Parameters:
     • state_code (string) (required)
     • park_code (string) (required)
     • query (string) (required)
     • limit (integer) (required)
----------------------------------------

2. get_park_alerts
   Description: Get current alerts for a specific national 

You can see the full output from the call to the Responses API above asking "Tell me about some events at some parks in Rhode Island.".  Here are some of the highlights:

* Output Item 1 shows that it called the NPS server to list all of that server's tools.
* Output Item 2 shows that it then called the `search_parks` tool from that server with arguments `{"state_code":"RI","limit":5}` and got back a list of 4 national parks in Rhode Island.  Since the limit it specified was 5 and it only got 4, presumably that's all there are in that state.
* Output Item 3 shows that it called `get_park_events` with arguments `{"park_code": "blrv", "limit": 5}`.  Notice that the value for `park_code` is the same as the value for `code` in the first output from the `search_parks` tool.  So the model recognized that the `code` field in the output of `search_parks` corresponds to the `park_code` field in the input of `get_park_events`.  It gets back a list of events for that park.
* Output Items 4-6 also call `get_park_events` with the other three park codes that had been returned by the `search_parks` tool.
* Output Item 7 then provides the actual response to be sent to the user.  It describes the outputs of all four of the calls to `get_park_events` in a human-friendly form.

One important thing to note is that all 7 output items came from *one* call to the Responses API.  This illustrates one of the advantages of this API and its implementation in Llama Stack: Llama Stack handles all of the coordination between all of these steps and calls the model and to the MCP server.  You could accomplish the same thing using basic "completions" API that allow you to specify tools, but it would then be up to you in the client to do all that coordinating.

## Using the OpenAI client

In the examples above we used the Llama Stack client to connect to Llama Stack.  However, since we're calling an OpenAI-compatible API (Responses), we can also use the OpenAI client to do the same thing.  The setup is a little bit clunkier than it is using the Llama Stack client.  With the OpenAI client, you have to add some extra stuff at the end of the URL to point the client at the OpenAI-compatible part of the overall Llama Stack.  However, once you do that, you can do the same things with the OpenAI client as we did with the Llama Stack client above.

Using the OpenAI client may be the best fit if you are writing code for a project that already uses that client for other purposes and you don't want to add any more dependencies to that project.

We start with the slightly clunky setup:

In [16]:

from openai import OpenAI

# Direct OpenAI client instantiation with Llama Stack
openai_client = OpenAI(
    api_key="no-key-needed",  # Llama Stack typically doesn't require a real key
    base_url=LLAMA_STACK_URL + "v1/openai/v1", # This suffix gets to the part of the URL that is specific to the OpenAI API, which you need here since you are using the OpenAI client.
)

Notice that while this is the OpenAI client, the URL we pointed it to is our Llama Stack server.  So even though we're using a different client here, we're still using the same server to handle the request.  Here we see that the simple example we used above works the same way as it did with the Llama Stack client:

In [17]:

simple_openai_client_response = openai_client.responses.create(
    model=LLAMA_STACK_MODEL_ID,
    input="What is the capital of France?"
)

print_response(simple_openai_client_response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"


ID: resp-1414198f-ad21-4608-9485-2b7a56f0da94
Status: completed
Model: gpt-4o
Created at: 1755107186.0
Output items: 1
Output type: message
Response content: The capital of France is Paris.


The RAG example also works the same way it did with the Llama Stack client:

In [18]:
import uuid

openai_client_vector_store_name= f"vec_{str(uuid.uuid4())[0:8]}"

openai_client_vector_store = openai_client.vector_stores.create(name=vector_store_name)
openai_client_vector_store_id = vector_store.id

openai_client_vector_store_id

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/vector_stores "HTTP/1.1 200 OK"


'vs_3bd59d59-2706-4165-8a95-dd3599ab988c'

In [19]:
openai_client_file_create_response = openai_client.files.create(file=Path(pdf_path), purpose="assistants")
openai_client_file_create_response

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/files "HTTP/1.1 200 OK"


FileObject(id='file-b2e794c7d681499e8f9433701dfd7a7b', bytes=6540612, created_at=1755107187, filename='NPIndex2012-2016.pdf', object='file', purpose='assistants', status=None, expires_at=1786643187, status_details=None)

In [20]:
openai_client_file_ingest_response = openai_client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_create_response.id,
)
openai_client_file_ingest_response

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/vector_stores/vs_3bd59d59-2706-4165-8a95-dd3599ab988c/files "HTTP/1.1 200 OK"


VectorStoreFile(id='file-02da3aef43cf435f92e1b913bbd3ddc4', created_at=1755107187, last_error=None, object='vector_store.file', status='completed', usage_bytes=0, vector_store_id='vs_3bd59d59-2706-4165-8a95-dd3599ab988c', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=None, type='auto'))

In [21]:
rag_openai_client_response = openai_client.responses.create(
    model=LLAMA_STACK_MODEL_ID,
    input="When did the Bering Land Bridge become a national preserve?",
    tools=[
        {
            "type": "file_search",
            "vector_store_ids": [vector_store_id],
        }
    ]
)

rag_openai_client_response

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"


Response(id='resp-29c11b0e-bd8a-442c-96d2-7427620e4a74', created_at=1755107214.0, error=None, incomplete_details=None, instructions=None, metadata=None, model='gpt-4o', object='response', output=[ResponseFileSearchToolCall(id='call_DYaV5m5WWhcVa5jJHNiF5pJI', queries=['Bering Land Bridge national preserve designation date'], status='completed', type='file_search_call', results=[Result(attributes=None, file_id='', filename='', score=3.0400664069376475, text=' for abundant wildlife and sport\n \nfishing for five species of salmon.\n \nEstablished Dec. 2, 1980. Length: 67 miles.\n \nAcreage—30,664.79 Federal: 26,417.85 \nNonfederal: 4,246.94. \nAniakchak  \nN\national Monument  and \nAniakchak  \nNational Preserve \n1000 Silver Street, Bldg.603 \nP\nO Box 245 \nKing Salmon, AK 99613 \n907\xad246\xad3305 \nwww.nps.gov/ania \nThe Aniakchak Caldera, covering some 30 square miles, is \no\nne of the great dry calderas in the world. Located in the \nvolcanically active Aleutian Mountains, the An

In [22]:
print_response(rag_openai_client_response)

ID: resp-29c11b0e-bd8a-442c-96d2-7427620e4a74
Status: completed
Model: gpt-4o
Created at: 1755107214.0
Output items: 2

--- Output Item 1 ---
Output type: file_search_call
  Tool Call ID: call_DYaV5m5WWhcVa5jJHNiF5pJI
  Tool Status: completed
  Queries: Bering Land Bridge national preserve designation date
  Results: [Result(attributes=None, file_id='', filename='', score=3.0400664069376475, text=' for abundant wildlife and sport\n \nfishing for five species of salmon.\n \nEstablished Dec. 2, 1980. Length: 67 miles.\n \nAcreage—30,664.79 Federal: 26,417.85 \nNonfederal: 4,246.94. \nAniakchak  \nN\national Monument  and \nAniakchak  \nNational Preserve \n1000 Silver Street, Bldg.603 \nP\nO Box 245 \nKing Salmon, AK 99613 \n907\xad246\xad3305 \nwww.nps.gov/ania \nThe Aniakchak Caldera, covering some 30 square miles, is \no\nne of the great dry calderas in the world. Located in the \nvolcanically active Aleutian Mountains, the Aniakchak last \nerupted in 1931. The crater includes lava flo

And similarly, the complex example using the NPS MCP server also works the same way as it did with the Llama Stack client:

In [23]:
complex_openai_client_response = openai_client.responses.create(
    model=LLAMA_STACK_MODEL_ID,
    input="Tell me about some parks in Rhode Island, and let me know if there are any upcoming events at them.",
    tools=[
        {
            "type": "mcp",
            "server_url": NPS_MCP_URL,
            "server_label": "National Parks Service tools",
        }
    ]
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"


Because both the OpenAI client and the Llama Stack client are wrapping the same underlying server API, the structures of their outputs wind up being the same too, so the complicated print_response method that prints out the response objects for the Llama Stack client also prints out the response objects for the OpenAI client.

In [24]:
print_response(complex_openai_client_response)

ID: resp-46a7faf0-9554-4aa4-9ed5-c15da5383c32
Status: completed
Model: gpt-4o
Created at: 1755107216.0
Output items: 7

--- Output Item 1 ---
Output type: mcp_list_tools

🔧 MCP Server: National Parks Service tools
   ID: mcp_list_cb8e98aa-881c-4e7e-8cbf-64fb7e96dff3
   Available Tools: 5

1. search_parks
   Description: Search for national parks by state, park code, or query string.

Args:
    state_code: Two-letter state code (e.g., 'CA', 'NY')
    park_code: Four-letter park code (e.g., 'yell', 'acad')
    query: Search query for park names or descriptions
    limit: Maximum number of results to return (default: 10)

Returns:
    JSON string with park information including name, description, website, and location
   Parameters:
     • state_code (string) (required)
     • park_code (string) (required)
     • query (string) (required)
     • limit (integer) (required)
----------------------------------------

2. get_park_alerts
   Description: Get current alerts for a specific nationa

As you can see, we get the same results here that we did with the Llama Stack client, as you would expect since we're calling the same Llama Stack server with the same arguments.

## LangChain

Both of the examples above are using clients that directly mirror the Responses API.  That means you have complete control over and visibility into exactly what's going into the API and exactly what's coming out of the API.  That can be useful and powerful, but it can also be a little complicated and tedious.  A lot of users prefer to use a framework that provides higher level abstractions.  Examples of such frameworks include LangChain, LangGraph, LlamaIndex, Haystack, and many more.

Here we configure LangChain with `use_responses_api` so it will use the same Responses API that we use above.  However, it hides a lot of the details of that API making the code simpler.

In [25]:
from langchain_openai import ChatOpenAI

# Pointing to your local Responses API server
llm = ChatOpenAI(
    model=LLAMA_STACK_MODEL_ID,
    base_url=LLAMA_STACK_URL + "v1/openai/v1", # This suffix gets to the part of the URL that is specific to the OpenAI API, which you need here since you are using the OpenAI client.,
    use_responses_api=True
)

In [26]:
langchain_simple_response = llm.invoke("What is the capital of France?")
print(langchain_simple_response.content)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"


[{'type': 'text', 'text': 'The capital of France is Paris.', 'annotations': []}]


There is a lot more that can be done with LangChain and packages like langchain_mcp_adapters to allow you to use MCP tools with LangChain, but a deeper dive into LangChain is beyond the scope of this document.  The key point here is that if you want a higher level framework with more abstract concepts, LangChain can be a good fit for that purpose, and you can point it at your Llama Stack server and have it use the Responses API.